In [1]:
from contracting.stdlib.bridge.hashing import sha3
import secrets

In [2]:
def rps_contract():
    # This tells the blockchain to make space for data. Variable and Hash are not Python builtins. 
    # They are globals (globals are something you can access without importing it) made available by the contracting. 
    # Calling Variable() creates a new piece of memory within the blockchain, that can hold a single value. 
    # If we say "foo = Variable()" then we can interact with that piece of memory through "foo". 
    # We can set the piece of memory to 1, for example by calling "foo.set(1)".
    # Setting the value will change the value for the next call to the smart contract.
    # We can get the piece of memory by calling "foo.get()".
    next_game_id = Variable()
    
    game_id_to_password_hash = Hash()
    game_id_to_player1_choice = Hash()
            
    # @construct means that this is the function that will be called ONCE when the smart contract is created
    @construct
    def constructor():
        # Game id starts at 0.
        next_game_id.set(0)
        
    # @export makes this function public, so it can be called by anyone on a deployed contract    
    @export
    def start_game(password_hash, player1_choice_hash):
        # This retrieves unique game Id from the blockchain. 
        unique_game_id = next_game_id.get()
        # This increments the number by 1, so the next game has a unique Id.
        next_game_id.set(next_game_id.get() + 1)
        
        # Hash throws error on integer keys. That's why we convert to a string
        # Remember player1 choice hash, and password hash for this game. 
        game_id_to_password_hash[str(unique_game_id)] = password_hash
        game_id_to_player1_choice[str(unique_game_id)] = player1_choice_hash
        
        return unique_game_id
    
    @export
    def submit_choice(game_id, game_password, player2_choice_hash):
        #We need to check that this game exists.
        assert game_id < next_game_id.get(), 'Game with id {} does not exist.'.format(game_id)
        #Now we need to check that this is the right password for the game.
        assert sha3(game_password) == game_id_to_password_hash[str(game_id)], 'Wrong password!'
        return
        
        
from contracting.client import ContractingClient
client = ContractingClient(signer='ren')

client.submit(rps_contract)

contract = client.get_contract('rps_contract')

Alice (player 1) chooses single use password. Only the person that has the password can join the game and play with Alice. Everything that starts with alice_ is only visible to Alice. 

In [3]:
alice_game_password = "trollbridge"

Alice hashes the password so she can submit it to the blockchain without sharing the actual password. She does this because everything on the blockchain is public, and she wants only the person she chooses to play the game with to have the password. 

*explain hashing

In [4]:
alice_game_password_hash = sha3(alice_game_password)
alice_game_password_hash

'47a5bcfb0d81053f5025ab57e6b94f43751f91bdb16fc0d63595223dc78ec1b4'

Alice chooses rock.

In [5]:
alice_choice = "rock"

We can't submit the choice to the blockchain as plain text, because then Bob (player 2) can see it and win by choosing paper. 

*explain blockchain sequentiality

In [6]:
alice_choice_hash = sha3(alice_choice)
alice_choice_hash

'bd996e2dc82a7c3e2c2da41291648e852b62b01fb09bcb6ed77975c273f08404'

The problem with subiting Alices choice like this is the 3 choices will be hashed the same everytime. Bob (player 2) can know what each of the hashes for the 3 choices and pick paper to win. To fix this Alice needs to pick a random salt to hash with her choice so that Bob can't guess her choice by looking at the hash.

In [7]:
alice_choice_salt = secrets.token_hex(32)
alice_choice_salt

'92ada16aad7bb63aff741055c12ff7e677981964ad3f0ee5e0b73633dae2a734'

In [8]:
alice_salted_choice = alice_choice + alice_choice_salt
alice_salted_choice

'rock92ada16aad7bb63aff741055c12ff7e677981964ad3f0ee5e0b73633dae2a734'

In [9]:
alice_salted_choice_hash = sha3(alice_salted_choice)
alice_salted_choice_hash

'2b69aa929da379bda48ef8a42599056e7c87fd0b472d0d1ad7572c8ddae50740'

Now Alice starts a game so she can invite Bob to play.

In [10]:
alice_game_id = contract.start_game(password_hash=alice_game_password_hash, player1_choice_hash=alice_salted_choice_hash) 
alice_game_id

0

Alice gets back a game Id.

Now Alice has to tell Bob the password and the game Id. This could be done over a messanger or built into the front end of an application. 

Everything that starts with bob_ is only visible to Bob.

In [11]:
bob_game_password = alice_game_password
bob_game_id = alice_game_id

Now it is Bobs turn.
Bob chooses scissors.

In [12]:
bob_choice = "scissors"

And now Bob has to salt his choice.

In [13]:
bob_choice_salt = secrets.token_hex(32)
bob_choice_salt

'6240c949a925e249b699213368b725f0a2e4e40e7ac01811676a2cd64751a417'

And now we combine bob_choice and bob_choice_salt together and hash them.

In [14]:
bob_salted_choice = bob_choice + bob_choice_salt
bob_salted_choice

'scissors6240c949a925e249b699213368b725f0a2e4e40e7ac01811676a2cd64751a417'

In [15]:
bob_salted_choice_hash = sha3(bob_salted_choice)
bob_salted_choice_hash

'6620491f304ab53ebd422d08d0e6ad75c60d18b90dc80ef46f8b5fd06ff860a8'

Bob now needs to submit his choice to the blockchain. Only Bob has the game password so only Bob can join Alices game.

In [16]:
contract.submit_choice(game_password=bob_game_password, game_id=bob_game_id, player2_choice_hash=bob_salted_choice_hash)

name 'sha3' is not defined


NameError: name 'sha3' is not defined